### Get raw data as a dataframe

In [63]:
import pandas as pd

df = pd.read_csv('transactions.csv')


df.head(2)

,acct_ccy,acct_id,bal_aftr_bookg,bal_aftr_bookg_nmrc,bookg_amt,bookg_amt_nmrc,bookg_cdt_dbt_ind,bookg_dt_tm_cet,bookg_dt_tm_gmt,booking_id,...,ctpty_ctry,ctpty_nm,dtld_tx_tp,end_to_end_id,ntry_seq_nb,rmt_inf_ustrd1,rmt_inf_ustrd2,tx_acct_svcr_ref,tx_tp,year_month
0,CAD,CA81VANC5752593800,1984.0,198400,74.0,7400,CRDT,2014-01-01 09:00:01,2014-01-01 09:00:01,8b8265bd-7ca4-47fe-bd44-ec3eb2870edb,...,NL,Geldmaat ATM,2222,72fed4e9-07ce-4e21-ae41-ff1aab7041fd,41517,Geldmaat: ea8bb272-9987-4ee5-93f3-870892f3e38d,NaN,3459cdd2-a497-4a86-9b44-a7182b06edd5,NaN,201401
1,EUR,NL12FAKE6711579483,5617.0,561700,4489.0,448900,CRDT,2014-01-01 09:00:05,2014-01-01 09:00:05,cee56886-53cf-43de-bca0-fc81434ee5ef,...,NL,Geldmaat ATM,2222,a9d48873-9b8c-4b33-a059-549eeb491ef1,753157,Geldmaat: 6ffbd0ab-610f-4435-852b-75ce8ac6a542,NaN,c3e7a151-0879-4de2-ac21-c6b9b27b7784,NaN,201401


### Get only necessary columns

In [64]:
necessary_columns_df = df.loc[:, ['acct_ccy', 'bookg_amt', 'ctpty_ctry', 'dtld_tx_tp']]

necessary_columns_df.head(4)

,acct_ccy,bookg_amt,ctpty_ctry,dtld_tx_tp
0,CAD,74.0,NL,2222
1,EUR,4489.0,NL,2222
2,EUR,1533.0,NL,2222
3,GBP,3953.0,France,2222


### Rename columns with a meaningful name

In [65]:
renamed_necessary_columns_df = necessary_columns_df.rename(
    columns={
        'acct_ccy': 'Currency', 
        'bookg_amt': 'Transaction Amount', 
        'ctpty_ctry': 'Country', 
        'dtld_tx_tp': 'Transaction Type'
    }
)

renamed_necessary_columns_df.head(4)

,Currency,Transaction Amount,Country,Transaction Type
0,CAD,74.0,NL,2222
1,EUR,4489.0,NL,2222
2,EUR,1533.0,NL,2222
3,GBP,3953.0,France,2222


### Get only EUR values from acct_ccy column

In [66]:
eur_rows = renamed_necessary_columns_df[renamed_necessary_columns_df['Currency'] == 'EUR']

eur_rows.head()

,Currency,Transaction Amount,Country,Transaction Type
1,EUR,4489.0,NL,2222
2,EUR,1533.0,NL,2222
4,EUR,2296.0,Great Britain,2222
5,EUR,408.0,NL,2222
7,EUR,4014.0,Austria,2222


### Get only NL values from Country column

In [67]:
nl_rows = eur_rows[eur_rows['Country'] == 'NL']

nl_rows.head()

,Currency,Transaction Amount,Country,Transaction Type
1,EUR,4489.0,NL,2222
2,EUR,1533.0,NL,2222
5,EUR,408.0,NL,2222
9,EUR,2591.0,NL,2222
10,EUR,1810.0,NL,2222


### Create a new column after Transaction Type column for category names

In [68]:
transaction_type_mapping = {
    5411: 'Groceries',
    5420: 'Food & Drinks',
    5462: 'Bakeries',
    5422: 'Freezer, Locker-meat provisioners',
    5912: 'Package stores, beer, wine, and liquor',
    4131: 'Buslines',
    4111: 'Transportation, suburban and local commuter passenger',
    4112: 'Passenger railways (long range)',
    5541: 'Service stations (with or without extra services)',
    5542: 'Fuel dispenser automated',
    4511: 'Aircarriers, airlines, not elsewhere classified',
    7512: 'Automobile rental agency, not elsewhere classified',
    4121: 'Limousines, taxi, cabs',
    6542: 'Gas & Electricity',
    6630: 'Water',
    3333: 'Tikkie',
    2222: 'ATM',
    1111: 'Income'
}

nl_rows.loc[:, 'Transaction Category'] = nl_rows['Transaction Type'].map(transaction_type_mapping)

nl_rows.head(100)

C:\Users\cagla\AppData\Local\Temp\ipykernel_28580\438607947.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nl_rows.loc[:, 'Transaction Category'] = nl_rows['Transaction Type'].map(transaction_type_mapping)


,Currency,Transaction Amount,Country,Transaction Type,Transaction Category
1,EUR,4489.00,NL,2222,ATM
2,EUR,1533.00,NL,2222,ATM
5,EUR,408.00,NL,2222,ATM
9,EUR,2591.00,NL,2222,ATM
10,EUR,1810.00,NL,2222,ATM
...,...,...,...,...,...
185,EUR,1460.53,NL,4511,"Aircarriers, airlines, not elsewhere classified"
188,EUR,105.29,NL,6542,Gas & Electricity
189,EUR,82.10,NL,7512,"Automobile rental agency, not elsewhere classi..."
190,EUR,181.95,NL,6542,Gas & Electricity


### Create a new column after category names for CO2 multipliers for NL

In [69]:
transaction_category_mapping = {
    'Groceries': '610,44',
    'Food & Drinks': '467,51',
    'Bakeries': '84,40',
    'Freezer, Locker-meat provisioners': '610,44',
    'Package stores, beer, wine, and liquor': '467,51',
    'Buslines': '226,24',
    'Transportation, suburban and local commuter passenger': '226,24',
    'Passenger railways (long range)': '255,74',
    'Service stations (with or without extra services)': '1175,43',
    'Fuel dispenser automated': '291,81',
    'Aircarriers, airlines, not elsewhere classified': '1666,78',
    'Automobile rental agency, not elsewhere classified': '235,11',
    'Limousines, taxi, cabs': '226,24',
    'Gas & Electricity': '592,94',
    'Water': '1045,62',
    'Tikkie': '74,03',
    'ATM': '74,03',
    'Income': '291,81'
}

nl_rows.loc[:, 'Transaction CO2 Multipliers'] = nl_rows['Transaction Category'].map(transaction_category_mapping)

nl_rows.head(100)

C:\Users\cagla\AppData\Local\Temp\ipykernel_28580\4221309049.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nl_rows.loc[:, 'Transaction CO2 Multipliers'] = nl_rows['Transaction Category'].map(transaction_category_mapping)


,Currency,Transaction Amount,Country,Transaction Type,Transaction Category,Transaction CO2 Multipliers
1,EUR,4489.00,NL,2222,ATM,"74,03"
2,EUR,1533.00,NL,2222,ATM,"74,03"
5,EUR,408.00,NL,2222,ATM,"74,03"
9,EUR,2591.00,NL,2222,ATM,"74,03"
10,EUR,1810.00,NL,2222,ATM,"74,03"
...,...,...,...,...,...,...
185,EUR,1460.53,NL,4511,"Aircarriers, airlines, not elsewhere classified","1666,78"
188,EUR,105.29,NL,6542,Gas & Electricity,"592,94"
189,EUR,82.10,NL,7512,"Automobile rental agency, not elsewhere classi...","235,11"
190,EUR,181.95,NL,6542,Gas & Electricity,"592,94"


### Create a new column after multipliers for TransactionToCO2 value

In [70]:
nl_rows['Transaction CO2 Multipliers'] = nl_rows['Transaction CO2 Multipliers'].str.replace(',', '.')

nl_rows['Transaction CO2 Multipliers'] = nl_rows['Transaction CO2 Multipliers'].astype(float)

nl_rows['TransactionToCO2'] = nl_rows['Transaction Amount'] * nl_rows['Transaction CO2 Multipliers']

nl_rows['TransactionToCO2'] = nl_rows['TransactionToCO2'].round(2)

nl_rows.head(100)

C:\Users\cagla\AppData\Local\Temp\ipykernel_28580\3756313596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nl_rows['Transaction CO2 Multipliers'] = nl_rows['Transaction CO2 Multipliers'].str.replace(',', '.')
C:\Users\cagla\AppData\Local\Temp\ipykernel_28580\3756313596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nl_rows['Transaction CO2 Multipliers'] = nl_rows['Transaction CO2 Multipliers'].astype(float)
C:\Users\cagla\AppData\Local\Temp\ipykernel_28580\3756313596.py:5: SettingWithCopyWarning:

,Currency,Transaction Amount,Country,Transaction Type,Transaction Category,Transaction CO2 Multipliers,TransactionToCO2
1,EUR,4489.00,NL,2222,ATM,74.03,332320.67
2,EUR,1533.00,NL,2222,ATM,74.03,113487.99
5,EUR,408.00,NL,2222,ATM,74.03,30204.24
9,EUR,2591.00,NL,2222,ATM,74.03,191811.73
10,EUR,1810.00,NL,2222,ATM,74.03,133994.30
...,...,...,...,...,...,...,...
185,EUR,1460.53,NL,4511,"Aircarriers, airlines, not elsewhere classified",1666.78,2434382.19
188,EUR,105.29,NL,6542,Gas & Electricity,592.94,62430.65
189,EUR,82.10,NL,7512,"Automobile rental agency, not elsewhere classi...",235.11,19302.53
190,EUR,181.95,NL,6542,Gas & Electricity,592.94,107885.43
